In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import glob
import json
import seaborn as sns
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
#from src.utils.logger_functions import console
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
import logging
import pickle
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

In [ ]:
MAIN_PATH = f'/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart'
#metadata = load_metadata_as_dataframe(file_path = os.path.join(MAIN_PATH, ''))

In [ ]:
PAIRINGS_PATH = os.path.join(MAIN_PATH, f'biTE_new/')
OUTPUT_PATH = os.path.join(MAIN_PATH,f'Results')
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

Merge pairs

In [ ]:
pickle_files = glob.glob(os.path.join(PAIRINGS_PATH, '*.pkl'))
pickle_files


In [ ]:
results_combined = pd.DataFrame()
for pickle_file in pickle_files:
    filename = os.path.basename(pickle_file)
    print(filename)
    filename_parts = filename.split('_')
    chip_id = int(filename_parts[0].replace('ID', ''))
    div = int(filename_parts[2].replace('DIV', ''))
    area = filename_parts[1]

    data_te = np.load(pickle_file, allow_pickle=True)

    try:    

        filtered_data = data_te['validated_results']
        #print(filtered_data)

        for pair in filtered_data:
            if pair['validation'] == 'good':

            #make one big dataframe containing filename, chip_id, div, area, syn probability
                results_combined = pd.concat([results_combined, pd.DataFrame({
                    'filename': filename,
                    'chip_id': [chip_id],
                    'div': [div],
                    'area': [area],
                    'syn_probability': [pair['syn probability']],
                    'mTE': [pair['mTE']],
                    'lag': [pair['lag']],
                    'source_electrode': [pair['source_electrode']],
                    'target_electrode': [pair['target_electrode']],
                    'source_unit_id': [pair['source_unit_id']],
                    'target_unit_id': [pair['target_unit_id']]
                })], ignore_index=True)
    except KeyError:
        print(f"KeyError: 'validated_results' not found in {filename}")
        continue
with open(os.path.join(OUTPUT_PATH, 'results_combined_probabilites.pkl'), 'wb') as f:
    pickle.dump(results_combined, f)

In [ ]:
with open(os.path.join(OUTPUT_PATH, 'results_combined_probabilites.pkl'), 'rb') as f:
    results_combined = pickle.load(f)

In [ ]:
def multi_dataframe_lineplot(data_list, x_metrics, y_metrics, axes, labels, colors, fontsize=16, labelsize=14, xlim=None, ylim=None, save_path=None, log_scale=False):
    plt.rcParams['xtick.bottom'] = True    
    plt.rcParams['ytick.left'] = True    
    plt.rcParams['font.family'] = 'Arial'
    
    # Plot each dataframe with its own color
    for data, x_metric, y_metric, label, color in zip(data_list, x_metrics, y_metrics, labels, colors):
        sns.lineplot(data=data, x=x_metric, y=y_metric, color=color, markers=True, estimator="mean", 
                    err_style="bars", label=label, zorder=1, ax=axes, dashes=False, 
                    marker="o", ms=10, linewidth=0.5, errorbar=("ci", 95))
    
    # Set common axes properties
    axes.set_ylabel(labels[0], fontsize=fontsize)  # Using the first label for y-axis title
    axes.set_xlabel("Day In Vitro", fontsize=fontsize)    
    axes.grid(True, linestyle=':', alpha=0.7)    
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    #axes.legend()
    
    if log_scale:
        axes.set_xscale('log')  # Set x-axis to log scale
    

    if xlim is not None:
        axes.set_xlim(xlim)
    if ylim is not None:
        axes.set_ylim(ylim)
    
    if save_path:
        plt.savefig(os.path.join(save_path, f'{labels[0][:10]}.png'), dpi=300)
        plt.savefig(os.path.join(save_path, f'{labels[0][:10]}.pdf'), dpi=300)

Plot synaptic probabilities over time

In [ ]:

MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Pickle_files_1851'
OUTPUT_PATH = os.path.join(MAIN_PATH, f'Results/')
with open(os.path.join(OUTPUT_PATH, 'results_combined_probabilites.pkl'), 'rb') as f:
   df_1 = pickle.load(f)
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/March2025_heart'
OUTPUT_PATH = os.path.join(MAIN_PATH, f'Results/')
with open(os.path.join(OUTPUT_PATH, 'results_combined_probabilites.pkl'), 'rb') as f:
   df_2 = pickle.load(f)

In [ ]:
# Filter data if needed (like in your example with speeds)
df_1 = pd.DataFrame(df_1)
df_2 = pd.DataFrame(df_2)
df_1 = df_1[df_1['syn_probability'] > 0.05]
df_2 = df_2[df_2['syn_probability'] > 0.05]
df_2 = df_2[df_2['div'] > 10]
df_1 = df_1[df_1['div'].isin([23, 65, 107, 129, 147])]

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Define colors - dimgray for first and a lighter gray for second
colors = ['dimgray', '#A9A9A9']  # dimgray and darkgray

# Call the modified function with both dataframes
multi_dataframe_lineplot(
    data_list=[df_1, df_2],
    x_metrics=[df_1['div'], df_2['div']],
    y_metrics=[df_1['syn_probability']*100, df_2['syn_probability']*100],
    axes=ax,
    labels=['Postsynaptic Spike Probability [%]', 'Postsynaptic Spike Probability [%]'],
    colors=colors,
    fontsize=16, 
    labelsize=14, 
    xlim=None, 
    ylim=(1, 60), 
    save_path=OUTPUT_PATH,
    log_scale=True
)

plt.tight_layout()